## Correction

In [ ]:
#!unzip '../data/tesseract.zip'

In [ ]:
#!unzip '../data/ground_truth.zip'

In [1]:
import glob
import os
import re
import string
from itertools import chain

truth = glob.glob(os.path.join('../data','ground_truth','*.txt'))
truth_clean = []
for x in truth:
    with open(x) as file:
        truth = file.readlines()
    truth = [x.strip() for x in truth] 
    truth = [x.lower() for x in truth]
    truth_str = ' '.join(truth)

    for i in range(10):
        truth_str = truth_str.replace(str(i),'')
    
    truth_clean.append(re.sub(r'[{}]'.format(string.punctuation), '', truth_str).split())
truth_clean = list(chain(*truth_clean))
print(len(truth_clean))
print(truth_clean[:20])


286240
['communications', 'network', 'member', 'companies', 'are', 'strongly', 'encouraged', 'to', 'provide', 'this', 'needed', 'support', 'the', 'state', 'advocacy', 'program', 'including', 'the', 'new', 'cmalinc']


In [2]:
teseract = glob.glob(os.path.join('../data','tesseract','*.txt'))
teseract_clean = []
for x in teseract:
    with open(x) as file:
        teseract = file.readlines()
    teseract = [x.strip() for x in teseract] 
    teseract = [x.lower() for x in teseract]
    teseract_str = ' '.join(teseract)

    for i in range(10):
        teseract_str = teseract_str.replace(str(i),'')
    
    teseract_clean.append(re.sub(r'[{}]'.format(string.punctuation), '', teseract_str).split())
teseract_clean = list(chain(*teseract_clean))
print(len(teseract_clean))
print(teseract_clean[:20])


283627
['communlcatlons', 'network', 'member', 'companles', 'are', 'strongly', 'encouraged', 'to', 'provlde', 'thls', 'needed', 'support', 'the', 'state', 'advocacy', 'program', 'ncludng', 'the', 'new', 'cmalinc']


In [3]:
import numpy as np
import pandas as pd
from collections import Counter 
from nltk import edit_distance

def typo_classification(typo,correct):
    if (len(typo) > len(correct)):
        return 'insertion'
    elif (len(typo) < len(correct)):
        return 'deletion'
    else:
        typo_count = Counter(typo)
        correct_count = Counter(correct)
        if typo_count == correct_count:
            return 'reversal'
        else:
            return 'substitution'

def find_candidates(typo,corpus):
    candidates = []
    candi_type = []
    for word in corpus:
        ed = edit_distance(typo,word)
        word_type = typo_classification(typo,word)
        if ((ed == 1) | ((ed == 2) & (word_type == 'reversal'))):
            candidates.append(word)
            candi_type.append(word_type)
    return candidates,candi_type

def find_position(typo,candidates):
    position = []
    for corr in candidates:
        typo_type = typo_classification(typo,corr)
        
        if (typo_type == 'deletion'):
            typo += '#'

            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    if corr[i] != corr[i-1]:
                        typo = typo[:-1]
                        position.append([typo,corr,"@",corr[i],i,typo_type])
                        break
                    else:
                        typo = typo[:-1]
                        position.append([typo,corr,"@",corr[i],i,typo_type])
                        position.append([typo,corr,"@",corr[i],i-1,typo_type])
                        break
                        
                i += 1
        elif (typo_type == 'insertion'):
            corr += '#'

            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    if typo[i] != typo[i-1]:
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        break
                    elif ((typo[i] == typo[i-1])& (typo[i] == typo[i-2])):
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        position.append([typo,corr,typo[i],"@",i-1,typo_type])
                        position.append([typo,corr,typo[i],"@",i-2,typo_type])
                        break
                    else:
                        corr = corr[:-1]
                        position.append([typo,corr,typo[i],"@",i,typo_type])
                        position.append([typo,corr,typo[i],"@",i-1,typo_type])
                        break
                i += 1
        elif (typo_type == 'substitution'):
            i = 0
            while i < len(corr):
                if (corr[i] != typo[i]):
                    position.append([typo,corr,typo[i],corr[i],i,typo_type])
                    break
                i+=1
                
        elif (typo_type == 'reversal'):
            i = 0
            while i < len(corr)-1:
                if ((typo[i] == corr[i+1]) & (typo[i+1] == corr[i])):
                    typo_comb = typo[i] + typo[i+1]
                    position.append([typo,corr,typo_comb,typo_comb[::-1],i,typo_type])
                    break
                i +=1
    return position

In [4]:
confusionsub=pd.read_csv('../data/confusion_matrix/sub_matrix.csv')
confusionadd=pd.read_csv('../data/confusion_matrix/add_matrix.csv')
confusiondel=pd.read_csv('../data/confusion_matrix/del_matrix.csv')
confusionrev=pd.read_csv('../data/confusion_matrix/rev_matrix.csv') 
corpus = set(truth_clean)

In [5]:
correction = pd.DataFrame()

def probabilityfunction(correction):
    for i in range(0,correction.shape[0]):
        index=correction.iloc[i,4]
        specificword=correction.iloc[i,1]
        if correction.iloc[i,5]=='insertion':
            if index != 0:

                #index=correction.iloc[i,4]
                X=specificword[index-1]
                Y=specificword[index]
                haha=confusionadd[confusionadd.iloc[:,0]==X]
                add=haha[Y].iloc[0]
                total=0
                for z in range(0,len(truth_clean)):
                    total=total+truth_clean[z].count(X+Y)
                    #lis.append(total)
                    correction.iloc[i,6]=add/total
            if index == 0:
                X='#'
                Y=specificword[index]
                haha=confusionadd[confusionadd.iloc[:,0]==X]
                add=haha[Y].iloc[0]
                totall=len(truth_clean)

                correction.iloc[i,6]=add/totall

        if correction.iloc[i,5]=='deletion':
            if index != 0:

                #index=correction.iloc[i,4]
                X=specificword[index-1]
                Y=specificword[index]
                haha=confusiondel[confusiondel.iloc[:,0]==X]
                delt=haha[Y].iloc[0]
                total=0
                for z in range(0,len(truth_clean)):
                    total=total+truth_clean[z].count(X+Y)
                    #lis.append(total)
                    correction.iloc[i,6]=delt/total


            if index == 0:
                X='#'
                Y=specificword[index]
                haha=confusiondel[confusiondel.iloc[:,0]==X]
                delt=haha[Y].iloc[0]
                totall=len(truth_clean)

                correction.iloc[i,6]=delt/totall
        if correction.iloc[i,5]=='reversal':


                #index=correction.iloc[i,4]
                X=specificword[index]
                Y=specificword[index+1]
                haha=confusionrev[confusionrev.iloc[:,0]==X]
                rev=haha[Y].iloc[0]
                total=0
                for z in range(0,len(truth_clean)):
                    total=total+truth_clean[z].count(X+Y)
                    #lis.append(total)
                    correction.iloc[i,6]=rev/total


        if correction.iloc[i,5]=='substitution':
            X=correction.iloc[i,2]
            Y=correction.iloc[i,3]
            heihei=confusionsub[confusionsub.iloc[:,0]==X]
            sub=heihei[Y].iloc[0]
            #lis.append(sub)

            total=0
            for z in range(0,len(truth_clean)):
                total=total+truth_clean[z].count(Y)
                #lis.append(total)
                correction.iloc[i,6]=sub/total
        correction['probability of t given c'] = correction.iloc[i,6]


In [6]:
typo =  pd.read_csv('cleaned_typos.csv') # Detection result(in list) 
typo = list(typo['n'])
typo = [x.lower() for x in typo]
wrong = pd.DataFrame(typo)


In [7]:
output = []

for typo in typo:
    
    candidates,cand_type = find_candidates(typo,corpus)
    correction = find_position(typo,candidates)
    correction = pd.DataFrame(correction)
    
    if correction.empty:  
        output.append(typo)
        
    else:
        correction.columns = ['Typo','Correction','old','new','index','type']
        
        # 1. calculate the prior
    
        freq = [] # the number of times that the proposed correction c appears in the training set
        for cor in correction['Correction']:
            freq.append(len(wrong[wrong[0] == cor]))    
    
        N = len(truth_clean) +len(teseract_clean) # the number of words
        vocabulary = truth_clean + teseract_clean
        V = len(set(vocabulary)) # the vocabulary size 

        prior = (pd.DataFrame(freq) + 0.5)/(N + V/2)
     
        correction['probability of c'] = prior

        # 2. Calculate the likelihood
        correction['probability of t given c']='defalut value'
        correction['probability of c']='defalut value'
        
        probabilityfunction(correction)
    
        # 3. Calculate the posterior and find the correction that has maximum posterior
        
        correction['posterior'] = correction['probability of c'] * correction['probability of t given c']
        p = correction[correction['posterior'] == max(correction['posterior'])]
        maxp = p['Correction'][p['Correction'].index[0]]
        output.append(''.join(maxp))
        

/Users/ying/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in long_scalars
/Users/ying/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: divide by zero encountered in long_scalars
/Users/ying/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in long_scalars
/Users/ying/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:78: RuntimeWarning: invalid value encountered in long_scalars


KeyError: '—'

In [ ]:
output

In [8]:
typo

'v'

In [9]:
correction

,Typo,Correction,old,new,index,type,probability of c,probability of t given c
0,v,i,v,i,0,substitution,0,0.0
1,v,vc,@,c,1,deletion,0,0.0
2,v,wv,@,w,0,deletion,8.38457e-05,0.0
3,v,f,v,f,0,substitution,9.27472e-05,0.0
4,v,r,v,r,0,substitution,0,0.0
5,v,vi,@,i,1,deletion,0.00691347,0.0
6,v,t,v,t,0,substitution,2.08459e-05,0.0
7,v,va,@,a,1,deletion,0.00651702,0.0
8,v,y,v,y,0,substitution,0,0.0
9,v,—,v,—,0,substitution,defalut value,0.0
